In [ ]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib

import os
import sys
import datetime
from pathlib import Path

#import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms

In [ ]:
from pytorch_modelsize import SizeEstimator

In [ ]:
#matplotlib.rcParams['figure.figsize'] = (12,7) # Til rapport
matplotlib.rcParams['figure.figsize'] = (20,10) # Til undervejs

## Vælg device

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.version.cuda)
print(device)

## Indlæs data
Det antages at dataen ligger i `'../data'`

In [ ]:
data_raw = np.load('../data/cullpdb+profile_5926.npy')

## Reshape data

In [ ]:
data = data_raw.reshape((-1,700,57))

In [ ]:
amino_acid_recidues  = data[...,:22]
amino_seq_profiles   = data[...,35:]
sec_structure_labels = data[...,22:31]
sec_structure_actual_labels = np.argmax(sec_structure_labels, axis=2).reshape((-1, 700, 1))
solvent_access       = data[...,33:35]

a = sec_structure_actual_labels.reshape((-1, 700, 1))
b = np.concatenate((a, solvent_access), axis=2)

ext_x = np.concatenate((amino_acid_recidues, amino_seq_profiles), axis=2)
ext_y = np.concatenate((sec_structure_actual_labels, solvent_access), axis=2)

x = ext_x
y = ext_y

input_channels  = x.shape[2]
output_channels = 11

print('Kanaler:\nInput:  %d\nOutput: %d' % (input_channels, output_channels))

print('Fuldt datasæt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

y_train_unrot = y[:5430]
y_test_unrot = y[5435:5690]
y_validation_unrot = y[5690:5926]

x = np.rot90(x, axes=(1,2))
y = np.rot90(y, axes=(1,2))

x = np.flip(x, 1)
y = np.flip(y, 1)

print('Fuldt datasæt vendt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

x_train = x[:5430]
y_train = y[:5430]

x_test = x[5435:5690]
y_test = y[5435:5690]

x_validation = x[5690:5926]
y_validation = y[5690:5926]

print('Splittet ud i training og testing:')
print('(Train) X: ', x_train.shape)
print('(Train) Y: ', y_train.shape)
print('(Test)  X: ', x_test.shape)
print('(Test)  Y: ', y_test.shape)
print('(Validation)  X: ', x_validation.shape)
print('(Validation)  Y: ', y_validation.shape)

torch_X_train = torch.from_numpy(x_train).type(torch.FloatTensor).to(device)
torch_Y_train = torch.from_numpy(y_train).type(torch.LongTensor).to(device)
torch_X_test  = torch.from_numpy(x_test).type(torch.FloatTensor).to(device)
torch_Y_test  = torch.from_numpy(y_test).type(torch.LongTensor).to(device)
torch_X_validation  = torch.from_numpy(x_validation).type(torch.FloatTensor).to(device)
torch_Y_validation  = torch.from_numpy(y_validation).type(torch.LongTensor).to(device)
print('Successfully moved data to device')

## Sæt data sammen i DataLoader

In [ ]:
BATCH_SIZE = 4

train = torch.utils.data.TensorDataset(torch_X_train, torch_Y_train)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=True)

## Definér modellen

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=input_channels,       
                out_channels=layer_widths[0],      
                kernel_size=kernel_sizes[0],        
                stride=1,             
                padding=int(kernel_sizes[0]/2),            
            ),                        
            nn.ReLU(),                
        )
        self.conv2 = nn.Sequential(   
            nn.Conv1d(
                in_channels=layer_widths[0],       
                out_channels=layer_widths[1],      
                kernel_size=kernel_sizes[1],        
                stride=1,             
                padding=int(kernel_sizes[1]/2),            
            ),                        
            nn.ReLU(),                
        )
        self.conv3 = nn.Sequential(   
            nn.Conv1d(
                in_channels=layer_widths[1],       
                out_channels=layer_widths[2],      
                kernel_size=kernel_sizes[2],        
                stride=1,             
                padding=int(kernel_sizes[2]/2),            
            ),                        
            nn.ReLU(),                
        )
        self.out = nn.Sequential(     
            nn.Conv1d(
                in_channels=layer_widths[2],       
                out_channels=11,       
                kernel_size=kernel_sizes[3],
                stride=1,             
                padding=int(kernel_sizes[3]/2),            
            ),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        output = self.out(x)
        secondary_structures = output[:,:-2,:]
        rel_solvent_acc = output[:,-2,:]
        abs_solvent_acc = output[:,-1,:]
        
        return secondary_structures, rel_solvent_acc, abs_solvent_acc 

## Instantiér modellen

In [ ]:
layer_width = 90
layer_widths = [layer_width] * 5
kernel_sizes = [5, 7, 9, 11]

torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)
cnn = CNN().to(device)
print(cnn)
print('Model is on device: "%s"' % device)

LR = 0.0005       # learning rate

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
sigge = nn.Sigmoid()

loss_func_structure = nn.CrossEntropyLoss()
loss_func_solvent   = nn.BCEWithLogitsLoss()
has_run = False

## Funktion til at måle accuracy

In [ ]:
def CalculateAccuracy(calc_values_structure, calc_values_rel, calc_values_abs, real_values):
    NoSeq = 8                                                                     # Hvilken værdi er padding?
    real_labels           = real_values[:,:,0]                                    # Split de korrekte værdier ud i sæt
    real_values_relsolv   = real_values[:,:,1]
    real_values_abssolv   = real_values[:,:,2]
    
    real_mask = real_labels == NoSeq                                              # Lav maske af dem der er NoSeq
    
    calc_values_structure = calc_values_structure.cpu().detach().numpy()          # Omform til numpy
    calc_values_structure = np.flip(calc_values_structure, 1)                     # Omgør spejlning og 
    calc_values_structure = np.rot90(calc_values_structure, k=-1, axes=(1,2))     # rotation
    
    calc_relsolv = sigge(calc_values_rel).cpu().detach().numpy()                  # Kør sigmoid funktion og omform til numpy
    
    calc_abssolv = sigge(calc_values_abs).cpu().detach().numpy()                  # Kør sigmoid funktion og omform til numpy
    
    calc_labels  = np.argmax(calc_values_structure, axis=2)                       # Kollaps one-hot til rene labels
    calc_relsolv = np.around(calc_relsolv)                                        # Afrund til enten 0 eller 1
    calc_abssolv = np.around(calc_abssolv)                                        # Afrund til enten 0 eller 1
    
    correct_structures = calc_labels == real_labels                               # Lav en matrice af korrekte forudsigelser
    correct_structures_masked = np.ma.masked_array(correct_structures, real_mask) # Filtrér dem er er NoSeq
    
    correct_relsolv = calc_relsolv == real_values_relsolv                         # Lav en matrice af korrekte forudsigelser
    correct_relsolv_masked = np.ma.masked_array(correct_relsolv, real_mask)       # Filtrér dem er er NoSeq
    
    correct_abssolv = calc_abssolv == real_values_abssolv                         # Lav en matrice af korrekte forudsigelser
    correct_abssolv_masked = np.ma.masked_array(correct_abssolv, real_mask)       # Filtrér dem er er NoSeq
    
    structure_mean = np.mean(correct_structures_masked)                           # Tag gennemsnittet af struktur-sættet
    relsolv_mean   = np.mean(correct_relsolv_masked)                              # Tag gennemsnittet af relativ solvent-sættet
    abssolv_mean   = np.mean(correct_abssolv_masked)                              # Tag gennemsnittet af absolut solvent-sættet
    
    return structure_mean, relsolv_mean, abssolv_mean

## Sammensat loss-funktion
Tager som input en loss-funktion (som i dette tilfælde er Binary Cross Entropy), tre beregnede matricer af hhv. sekundærstrukturer og relativ- og absolut solvent accessibility, anvender loss-funktionen på dem alle og returnerer summen.

In [ ]:
def handleLoss(calc_struct, calc_rel, calc_abs, correct, verbose=False):    
    correct_struct  = correct[:,0,:]
    correct_rel     = correct[:,1,:].type(torch.FloatTensor).to(device)
    correct_abs     = correct[:,2,:].type(torch.FloatTensor).to(device)
    
    loss1 = loss_func_structure(calc_struct,  correct_struct)
    loss2 = loss_func_solvent(calc_rel, correct_rel)
    loss3 = loss_func_solvent(calc_abs,  correct_abs)
    if verbose:
        print('Secondary structure loss: %.4f' % loss1.item())
        print('Relative solvent accessibility loss: %.4f' % loss2.item())
        print('Absolute solvent accessibility loss: %.4f' % loss3.item())
    
    loss_sum = (loss1 + loss2 + loss3)
    
    return loss_sum

## Arrays til at gemme data til visualisering

In [ ]:
rel_accs_v_multi = []
rel_accs_t_multi = []
abs_accs_v_multi = []
abs_accs_t_multi = []
acc_struct_v_multi = []
acc_struct_t_multi = []

steps        = []
steps_cum    = []
epochs       = []

## Træn modellen

In [ ]:
EPOCH = 10

if has_run:
    print('Du har ikke genstartet modellen')
    print(1/0)

has_run = True


time_start = datetime.datetime.now()

step_cum = -1
for epoch in range(EPOCH):
    print('\nEpoch: ', epoch+1)
    epochs.append(step_cum)
    for step, (b_x, b_y) in enumerate(train_loader):
        step_cum += 1
        a, b, c = cnn(b_x)
        optimizer.zero_grad()
        loss = handleLoss(a, b, c, b_y)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            v_a, v_b, v_c = cnn(torch_X_validation)
            vloss = handleLoss(v_a, v_b, v_c, torch_Y_validation)
            acc_struc_v, acc_rel_v, acc_abs_v = CalculateAccuracy(v_a, v_b, v_c, y_validation_unrot)
            
            t_a, t_b, t_c = cnn(torch_X_test)
            vloss = handleLoss(t_a, t_b, t_c, torch_Y_test)
            acc_struc_t, acc_rel_t, acc_abs_t = CalculateAccuracy(t_a, t_b, t_c, y_test_unrot)
            sys.stdout.write('\r%d\t%.4f\t%.4f\t%.4f\t%.4f\t%.4f\t%.4f\t' % (step, acc_struc_v*100, acc_rel_v*100, acc_abs_v*100, acc_struc_t*100, acc_rel_t*100, acc_abs_t*100))
            
            acc_struct_v_multi.append(acc_struc_v)
            acc_struct_t_multi.append(acc_struc_t)
            rel_accs_v_multi.append(acc_rel_v)
            rel_accs_t_multi.append(acc_rel_t)
            abs_accs_v_multi.append(acc_abs_v)
            abs_accs_t_multi.append(acc_abs_t)
            steps_cum.append(step_cum)
            
            
print('\nDone training.')
time_end = datetime.datetime.now()

v_a, v_b, v_c = cnn(torch_X_validation)
vloss = handleLoss(v_a, v_b, v_c, torch_Y_validation)
acc_struc_v, acc_rel_v, acc_abs_v = CalculateAccuracy(v_a, v_b, v_c, y_validation_unrot)

t_a, t_b, t_c = cnn(torch_X_test)
vloss = handleLoss(t_a, t_b, t_c, torch_Y_test)
acc_struc_t, acc_rel_t, acc_abs_t = CalculateAccuracy(t_a, t_b, t_c, y_test_unrot)
print('\tValidation\t\tTest\nsec\trel\tabs\tsec\trel\tabs')
sys.stdout.write('%.4f\t%.4f\t%.4f\t%.4f\t%.4f\t%.4f\n' % (acc_struc_v*100, acc_rel_v*100, acc_abs_v*100, acc_struc_t*100, acc_rel_t*100, acc_abs_t*100))

print(time_end - time_start)

In [ ]:
savefile = 0
filename = '../graphs/new/multi-task.png'

LW=2

plt.plot(steps_cum[3:], rel_accs_v_multi[3:], label='Relative solvency accuracy')#, lw=LW)
plt.plot(steps_cum[3:], abs_accs_v_multi[3:], label='Absolute solvency accuracy')#, lw=LW)
plt.plot(steps_cum[3:], acc_struct_v_multi[3:], label='Secondary structure accuracy')#, lw=LW)

for i in epochs:
    plt.axvline(x=i, alpha=0.25, c='g')

plt.title('Multi-task model, validation set')
plt.xlabel('Training steps')
plt.ylabel('Accuracy (%)')
plt.legend(prop={'size': 15})

if savefile:
    my_file = Path(filename)
    if my_file.is_file():
        print('Filen findes allerede')
    else:
        print('Gemmer som:', filename)
        plt.savefig(filename)


## Gem værdier
Gemmer værdierne så de kan hentes i visualization.ipynb

In [ ]:
%store rel_accs_v_multi
%store rel_accs_t_multi
%store abs_accs_v_multi
%store abs_accs_t_multi
%store acc_struct_v_multi
%store acc_struct_t_multi
%store epochs